In [3]:
import json
import pandas as pd

In [4]:
filepath = "../1. Data Scraping/output/data.json"

In [18]:
# Load JSON data
with open(filepath, "r") as file:
    data = json.load(file)

# Extracting data
rows = []
for date, details in data.items():
    x = details["aqi"]
    if( x != None ):
        if( x["value"] != -1):
            row = {"Date": date}  # Daily date
            for metric in details["metrics"]:

                row[metric["name"]] = metric["avg"]  # Daily avg pollutant values

            row["AQI"] = x["value"]
    rows.append(row)


# Convert to DataFrame
df = pd.DataFrame(rows)

# Save as CSV
df.to_csv("AQI_data.csv", index=False)

# Display the first few rows
df.head(50)


,Date,PM2.5,PM10,NO2,NH3,SO2,CO,OZONE,AQI
0,2019-05-01T00:00:00Z,81,140,13,2,28,22,23,140
1,2019-05-02T00:00:00Z,129,214,14,2,17,25,25,214
2,2019-05-03T00:00:00Z,89,144,11,2,17,13,43,144
3,2019-05-04T00:00:00Z,-,-,-,-,-,20,23,23
4,2019-05-05T00:00:00Z,78,116,6,2,13,27,25,116
5,2019-05-06T00:00:00Z,65,109,4,5,12,23,30,109
6,2019-05-07T00:00:00Z,62,105,6,4,13,34,8,105
7,2019-05-08T00:00:00Z,55,101,6,4,14,26,14,101
8,2019-05-09T00:00:00Z,58,104,8,2,34,22,21,104
9,2019-05-10T00:00:00Z,50,80,6,3,29,19,12,80
